<a href="https://colab.research.google.com/github/kamalkraj/minGPT-TF/blob/master/play_char.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kamalkraj/minGPT-TF/blob/master/play_char.ipynb)

In [1]:
!git clone https://github.com/kamalkraj/minGPT-TF.git

fatal: destination path 'minGPT-TF' already exists and is not an empty directory.


In [2]:
! pip install fastprogress==0.2.3

In [3]:
import os
os.chdir('minGPT-TF')

In [4]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2021-09-01 14:42:48--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2021-09-01 14:42:48 (45.9 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [5]:
import math
import numpy as np
import tensorflow as tf
from mingpt.model import GPT, GPTConfig

In [6]:
class CharDataset:

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return math.ceil(len(self.data) / (self.block_size + 1))

    def __iter__(self):
        # we're actually going to "cheat" and pick a spot in the dataset at random
        for _ in range(self.__len__()):
            i = np.random.randint(0, len(self.data) - (self.block_size + 1))
            chunk = self.data[i:i+self.block_size+1]
            dix = [self.stoi[s] for s in chunk]
            x = tf.convert_to_tensor(dix[:-1], dtype=tf.int32)
            y = tf.convert_to_tensor(dix[1:], dtype=tf.int32)
            yield x, y
    
    __call__ = __iter__

In [7]:
block_size = 128 

In [8]:
text = open('input.txt', 'r').read()
train_dataset_gen = CharDataset(text, block_size) 

data has 1115394 characters, 65 unique.


In [9]:
train_dataset = tf.data.Dataset.from_generator(train_dataset_gen,(tf.int32,tf.int32))

In [10]:
from mingpt.model import GPT, GPTConfig
mconf = GPTConfig(train_dataset_gen.vocab_size, train_dataset_gen.block_size,
                  n_layer=8, n_head=8, n_embd=512)

In [11]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=10, batch_size=64, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=200*len(train_dataset_gen)*block_size,
                      num_workers=4)
trainer = Trainer(GPT, mconf, train_dataset, len(train_dataset_gen), None, None, tconf)

In [12]:
trainer.train()

epoch 1: train loss 346.91553. lr 5.999636e-04
epoch 2: train loss 292.35272. lr 5.998533e-04
epoch 3: train loss 257.20963. lr 5.996690e-04
epoch 4: train loss 231.03250. lr 5.994107e-04
epoch 5: train loss 213.96826. lr 5.990785e-04
epoch 6: train loss 201.60358. lr 5.986725e-04
epoch 7: train loss 192.35039. lr 5.981929e-04
epoch 8: train loss 185.77501. lr 5.976396e-04
epoch 9: train loss 181.50128. lr 5.970130e-04
epoch 10: train loss 176.67290. lr 5.963130e-04


In [13]:
# alright, let's sample some character-level shakespear
from mingpt.utils import sample

context = "O God, O God!"
x = tf.convert_to_tensor([train_dataset_gen.stoi[s] for s in context], dtype=tf.int32)[None,...]
y = sample(trainer.model, x, 2000, temperature=0.9, sample=True, top_k=5)[0]
completion = ''.join([train_dataset_gen.itos[int(i)] for i in y])
print(completion)

O God, O God! save a strang anger to me, but not
A supper to be him a stand an attimation, he will
And the hanged to have may a pleased that would
A the high with thy cursed to make the cries here,
And what I have, that we may may be hum
That the placed of the mouth of her hated hate breath
A told me think the prince of him for him from thee,
The prize his foolish of his play's liege,
Wher we home homast to be much all my son.

KING LEWIS XI:
I'll not be succustor's to the maid had a poor
A solenger of truth of my servant, that I do news the sea
The horsely have seem of a mean to bless a cut an
The seat of the present and to this sake the prison,
That will be a streams of make than he seals
That I will dear a man, a then to her best.

KING RICHARD IIII:
Is thou answer thee o' the means of the mind
As the power his supose on a fool a must
To be him his consencent it on me,
Which are strong but answer all him, he stay is:
The wear one on the will to-morrow any then they say
The said was 